In [1]:
import pandas as pd
import numpy as np
from langdetect import detect
import re

In [2]:
labeled1 = pd.read_excel('manual_labels.xlsx')
labeled2 = pd.read_excel('manual_labels2.xlsx')
labeled3 = pd.read_excel('manual_labels3.xlsx')
labeled4 = pd.read_excel('manual_labels4.xlsx')

In [3]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import re
def clean_comment(text):
    wnl = WordNetLemmatizer()
    deacc = re.sub(r'\W',' ', text)
    tokens = word_tokenize(deacc)
    res = ''
    for t in tokens:
        res += wnl.lemmatize(t)+' '
    return res
def get_tokens(text):
    wnl = WordNetLemmatizer()
    deacc = re.sub(r'\W',' ', text)
    tokens = word_tokenize(deacc)
    return tokens

In [4]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import re
import nltk
from nltk.corpus import wordnet
def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return ''

def clean_comment(text):
    wnl = WordNetLemmatizer()
    #deacc = re.sub(r'\W',' ', text)
    tokens = word_tokenize(text)
    tags = nltk.pos_tag(tokens)
    processed = []
    for (word, tag) in tags:
        wn_tag = get_wordnet_pos(tag)
        if wn_tag!='':
            processed.append(wnl.lemmatize(word,wn_tag))
        else:
            processed.append(wnl.lemmatize(word))
    return ' '.join(processed)

In [5]:
labeled = pd.concat([labeled1, labeled2, labeled3, labeled4],axis = 0)

In [6]:
labeled.loc[:,'cleaned'] = labeled.Review.apply(clean_comment)

In [7]:
labeled['tokens'] = labeled.Review.apply(get_tokens)

In [8]:
labeled_long = labeled[labeled.tokens.apply(len)>6]
labeled_neg = labeled[labeled.label!=4]

In [9]:
comments = pd.read_csv('../data/comments_lang.csv')
comments_en = comments[comments.lang == 'en']
comments_en.loc[:,'cleaned'] = comments_en.Review.apply(clean_comment)

/home/lsm/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:357: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/home/lsm/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [12]:
labeled.tokens.apply(len).max()

273

In [35]:
from sklearn.feature_extraction.text import TfidfVectorizer

from keras.preprocessing import sequence
from keras.preprocessing.sequence import pad_sequences
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
#from keras.layers.cudnn_recurrent import CuDNNLSTM
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import TimeDistributed
from keras.models import load_model
from keras.utils import Sequence
from keras.preprocessing.text import Tokenizer
import keras.backend as K
import gensim
from sklearn.model_selection import train_test_split
from keras import regularizers
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
from sklearn.utils.class_weight import compute_class_weight

In [49]:
t = Tokenizer()
t.fit_on_texts(comments_en.cleaned.tolist())
vocab_size = len(t.word_index) + 1
# integer encode the documents
encoded_docs = t.texts_to_sequences(labeled.cleaned.tolist())
max_length = labeled.tokens.apply(len).max()
feats = encoded_docs
labels = to_categorical(labeled.label)

X_train, X_test, y_train, y_test = train_test_split(feats, labels, test_size=0.2)
X_train = sequence.pad_sequences(X_train, maxlen=max_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_length)
#X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
#X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

In [55]:
class_weight = compute_class_weight('balanced'
                                               ,[0,1,2,3,4,5]
                                               ,labeled.label.apply(int).tolist())
model = Sequential()
model.add(Embedding(vocab_size, 100, 
                    input_length=max_length,
                    #input_shape = (None,max_length),
                    embeddings_regularizer = regularizers.l1(1e-5)))
model.add(Dropout(0.7))
model.add(Conv1D(filters=700, kernel_size=2, padding='same', activation='tanh'))
#model.add(Conv1D(filters=7, kernel_size=2, padding='same', activation='tanh'))
#model.add(Conv1D(filters=50, kernel_size=3, padding='same', activation='sigmoid'))
model.add(MaxPooling1D(pool_size=2))
#model.add(Flatten())
model.add(Dropout(0.7))
model.add(LSTM(25, activation = 'tanh', kernel_regularizer = regularizers.l1(1e-5)))
model.add(Dropout(0.7))
model.add(Dense(6, activation='softmax'))
model.compile(loss='categorical_hinge', optimizer='adam', metrics=['categorical_accuracy'])

checkpointer = ModelCheckpoint(filepath='weights.hdf5',
                               verbose=1, save_best_only=True,
                               monitor = 'val_categorical_accuracy')
model.fit(X_train, y_train, epochs=50, batch_size=100,
          validation_data = [X_test,y_test], callbacks=[checkpointer],
         class_weight = class_weight)

Train on 1818 samples, validate on 455 samples
Epoch 1/50
1800/1818 [============================>.] - ETA: 0s - loss: 1.2654 - categorical_accuracy: 0.4161
Epoch 00001: val_categorical_accuracy improved from -inf to 0.49670, saving model to weights.hdf5
1818/1818 [==============================] - 23s 13ms/step - loss: 1.2645 - categorical_accuracy: 0.4164 - val_loss: 1.1641 - val_categorical_accuracy: 0.4967
Epoch 2/50
1800/1818 [============================>.] - ETA: 0s - loss: 1.1087 - categorical_accuracy: 0.4878
Epoch 00002: val_categorical_accuracy improved from 0.49670 to 0.50549, saving model to weights.hdf5
1818/1818 [==============================] - 21s 12ms/step - loss: 1.1080 - categorical_accuracy: 0.4879 - val_loss: 0.9972 - val_categorical_accuracy: 0.5055
Epoch 3/50
1800/1818 [============================>.] - ETA: 0s - loss: 0.9663 - categorical_accuracy: 0.5406
Epoch 00003: val_categorical_accuracy improved from 0.50549 to 0.59560, saving model to weights.hdf5
1818/

1800/1818 [============================>.] - ETA: 0s - loss: 0.3866 - categorical_accuracy: 0.8411
Epoch 00024: val_categorical_accuracy improved from 0.72967 to 0.72967, saving model to weights.hdf5
1818/1818 [==============================] - 21s 11ms/step - loss: 0.3850 - categorical_accuracy: 0.8421 - val_loss: 0.5784 - val_categorical_accuracy: 0.7297
Epoch 25/50
1800/1818 [============================>.] - ETA: 0s - loss: 0.3770 - categorical_accuracy: 0.8428
Epoch 00025: val_categorical_accuracy did not improve
1818/1818 [==============================] - 21s 12ms/step - loss: 0.3760 - categorical_accuracy: 0.8432 - val_loss: 0.5836 - val_categorical_accuracy: 0.7209
Epoch 26/50
1800/1818 [============================>.] - ETA: 0s - loss: 0.3750 - categorical_accuracy: 0.8494
Epoch 00026: val_categorical_accuracy did not improve
1818/1818 [==============================] - 21s 12ms/step - loss: 0.3737 - categorical_accuracy: 0.8498 - val_loss: 0.5909 - val_categorical_accuracy: 

Epoch 49/50
1800/1818 [============================>.] - ETA: 0s - loss: 0.3086 - categorical_accuracy: 0.8733
Epoch 00049: val_categorical_accuracy did not improve
1818/1818 [==============================] - 22s 12ms/step - loss: 0.3097 - categorical_accuracy: 0.8724 - val_loss: 0.6041 - val_categorical_accuracy: 0.7231
Epoch 50/50
1800/1818 [============================>.] - ETA: 0s - loss: 0.3059 - categorical_accuracy: 0.8678
Epoch 00050: val_categorical_accuracy did not improve
1818/1818 [==============================] - 21s 12ms/step - loss: 0.3053 - categorical_accuracy: 0.8685 - val_loss: 0.5978 - val_categorical_accuracy: 0.7231


In [56]:
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score

In [57]:
from sklearn.metrics import f1_score,roc_auc_score,accuracy_score
def eval_classifier(input_text,model = model):
    cleaned_text = clean_comment(input_text)
    encoded_doc = t.texts_to_sequences([cleaned_text])
    padded_doc = sequence.pad_sequences(encoded_doc, maxlen=max_length)
    feats = padded_doc#vect.transform(padded_doc)
    class_names = ['Other',
        'Crash',
        'Balance problems',
        'Synchronization',
        'Positive',
        'Bug']
    prediction = model.predict(feats)
    #print(class_names[prediction[0]])
    return np.argmax(prediction[0])
def val_score(model):
    class_names = ['Other',
        'Crash',
        'Balance problems',
        'Synchronization',
        'Positive',
        'Bug']    
    val_en = pd.read_excel('validation_en.xlsx')
    y_true = []
    y_pred = []
    for i in range(0,6):
        y_true.append([i]*10)
        y_pred.append(val_en.iloc[:,i].apply(eval_classifier))
    y_true = np.array(y_true).flatten()
    y_pred = np.array(y_pred).flatten()
    print(classification_report(y_true, y_pred, target_names = class_names))
    return y_true,y_pred
y_true,y_pred = val_score(model)

                  precision    recall  f1-score   support

           Other       0.00      0.00      0.00        10
           Crash       0.00      0.00      0.00        10
Balance problems       0.45      1.00      0.62        10
 Synchronization       0.00      0.00      0.00        10
        Positive       0.25      0.40      0.31        10
             Bug       0.27      0.60      0.37        10

     avg / total       0.16      0.33      0.22        60



/home/lsm/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [54]:
val_en = pd.read_excel('validation_en.xlsx')
val_en.Bug.apply(eval_classifier)

0    2
1    4
2    2
3    2
4    2
5    2
6    2
7    4
8    2
9    4
Name: Bug, dtype: int64